In [30]:
from cassandra.cluster import Cluster
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly
import json
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [3]:
cluster = Cluster(port=9042)
session = cluster.connect('crytocurrencies')

In [4]:
data = session.execute('SELECT * FROM bitcoin').all()
df = pd.DataFrame(data)

In [ ]:
# Prepare data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

prediction_days = 60
future_days = 5

x_train, y_train = [], []

for x in range(prediction_days, len(scaled_data)):
  x_train.append(scaled_data[x-prediction_days:x, 0])
  y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1], 1))

In [ ]:
# Create Neural Network 

model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

In [33]:
date = datetime.now().date()
test_data = session.execute(f"SELECT * FROM bitcoin WHERE datetime > '{date}' ALLOW FILTERING;").all()
test_data = pd.DataFrame(test_data)
test_data = test_data.sort_values(by='datetime')

In [ ]:
# Testing the model
actual_price = test_data['Close'].values
total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset)- len(test_data) - prediction_days:].values.reshape(-1, 1)
model_inputs = scaler.fit_transform(model_inputs)

x_test = []

for x in range(prediction_days, len(model_inputs)):
  x_test.append(model_inputs[x-prediction_days:x, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1], 1))

prediction_price = model.predict(x_test)
prediction_price = scaler.inverse_transform(prediction_price)

In [ ]:
fig = plt.figure()
plt.plot(actual_price, color='black', label='Actual Price')
plt.plot(prediction_price.flatten(), color='green', label='Prediction Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc='upper left')

fig.show()

In [ ]:


fig = go.Figure()

fig.add_trace(go.Scatter(x=test_data['Datetime'],
                           y=actual_price, 
                           name='Actual Price'))

fig.add_trace(go.Scatter(x=test_data['Datetime'],
                           y=prediction_price.flatten(), 
                           name='Predict Price'))

fig.update_layout(title='Bitcoin Prediction Price',
                    yaxis_title='Stock Price (USD per Shares)')

fig

In [ ]:
# Predict the day

real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs) + 1, 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)

print(f"Predict price of Bitcoin tomorrow: {prediction}")